## 2. Clean the data

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import pprint
import bnphelper
import csv

### 2.1. Data Preparation

In [59]:
pp = pprint.PrettyPrinter()

dataset_train = dataiku.Dataset("train")
dataset_test = dataiku.Dataset("test")

# Read the dataset as a Pandas dataframe
train_df = dataset_train.get_dataframe(limit=2000000)
test_df = dataset_test.get_dataframe(limit=2000000)

#### 2.1.1. Categorical & Continuous variables

In [50]:
# conver some numeric variables to categorical
bnphelper.change_numeric_to_categorical(train_df, 40)

train_audit = pdu.audit(train_df)
train_audit_predictors = train_audit.iloc[2:,:]
#print train_audit_predictors

# Get the numeric and categorical sets
#train_filter_numeric = (train_audit_predictors._b_data_type == 'int64') | (train_audit_predictors._b_data_type == 'float64')
train_filter_numeric = train_audit_predictors._b_data_type == 'float64'
train_audit_numeric = train_audit_predictors[train_filter_numeric]
train_audit_text = train_audit_predictors[-train_filter_numeric]

#print 'Number of numeric variables:', train_audit_numeric.shape
#pp.pprint(train_audit_numeric)
#print '\n'
print 'Number of categorical variables', train_audit_text.shape
pp.pprint(train_audit_text)

Number of categorical variables (23, 5)
   _a_variable _b_data_type  _c_cardinality  _d_missings _e_sample_values
4           v3       object               4         3457         [C, nan]
15         v22       object           18211          500       [XDX, GUV]
17         v24       object               5            0           [C, E]
20         v30       object               8        60110         [C, nan]
21         v31       object               4         3457           [A, B]
25         v38       object              12            0           [0, 4]
31         v47       object              10            0           [C, E]
35         v52       object              13            3           [G, F]
36         v56       object             123         6882         [DI, DY]
41         v62       object               8            0           [1, 2]
42         v66       object               3            0           [C, A]
46         v71       object               9            0           [F, B

In [61]:
# Testing
train_audit_text

,_a_variable,_b_data_type,_c_cardinality,_d_missings,_e_sample_values
4,v3,object,4,3457,"[C, nan]"
15,v22,object,18211,500,"[XDX, GUV]"
17,v24,object,5,0,"[C, E]"
20,v30,object,8,60110,"[C, nan]"
21,v31,object,4,3457,"[A, B]"
25,v38,object,12,0,"[0, 4]"
31,v47,object,10,0,"[C, E]"
35,v52,object,13,3,"[G, F]"
36,v56,object,123,6882,"[DI, DY]"
41,v62,object,8,0,"[1, 2]"


In [62]:
# MyTesting
train_df.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,C,0.139412,1.720818,3.393503,0.590122,8.880867,C,A,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,0,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,C,12.171733,8.086643,0.899420,7.277792,G,16.747968,0.037096,1.299638,DI,3.971118,0.529802,10.890984,1.588448,15.858152,1,0.153461,6.363189,18.303925,C,9.314079,15.231789,17.142857,11.784549,F,1,1.614988,B,D,2.230940,7.292418,8.571429,E,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,A,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,E,2.382692,3.930922,B,0.433213,O,NaN,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,C,3.056144,NaN,NaN,NaN,NaN,C,A,NaN,NaN,3.615077,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,E,NaN,NaN,1.379210,NaN,G,NaN,1.129469,NaN,DY,NaN,NaN,NaN,NaN,NaN,2,2.544736,NaN,NaN,A,NaN,NaN,NaN,12.053353,F,2,NaN,B,D,NaN,NaN,NaN,D,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,B,1.825361,4.247858,A,NaN,U,G,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,E,0.113997,2.244897,5.306122,0.836005,7.499999,NaN,A,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,0,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,C,12.711328,6.836734,0.604504,9.637627,F,15.102041,0.085573,0.765305,AS,4.030613,4.277456,9.105481,2.151361,16.075602,1,0.123643,5.517949,16.377205,A,8.367347,11.040463,5.882353,8.460654,B,3,2.413618,B,B,1.963971,5.918368,11.764705,E,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,G,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.670572,C,1.375753,1.184211,B,3.367348,S,NaN,11.205561,12.941177,3.129253,3.478911,6.233767,-2.792745e-07,2.138728,2.238806,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,16.347483,9.646653,3.903302,14.094723,1.945044,5.517242,3.610789,1.224114,0.231630,18.376407,7.517125,ACUE,-4.805344e-07,D,0.148843,1.308269,2.303640,8.926662,8.874521,C,B,1.587644,1.666667,8.703550,8.898468,11.302795,0.433735,0,0.287322,11.523045,7.931035,12.935823,1.470878,12.708574,9.670823,0.108387,C,12.194855,8.591954,3.329176,4.780357,H,16.621695,0.139721,1.178161,BW,3.965517,1.732102,11.777912,1.229246,15.927390,1,0.140260,6.292979,17.011645,A,9.703065,18.568129,9.425288,13.594728,F,2,2.272541,B,D,2.188198,8.213602,13.448277,B,1.947261,4.797873,13.315819,1.681034,1.379310,1.587045,1.242817,10.

### 2.2. Correlation

In [65]:
corr_var = train_df.iloc[2:,2:].corr()

In [66]:
### Correlation
correlation_threshold = 0.7

#print corr_var.columns

variables_na = []
high_correlation = []

for index, row in corr_var.iterrows():
    row_index_num = corr_var.index.get_loc(index)
    
    na_count = sum(train_df.iloc[:,row_index_num].isnull())
    
    row_has_na = train_df.iloc[:,row_index_num].isnull().any()
    
    if (row_has_na):
        variables_na.append(row.name)
    
    for colname in corr_var.columns.values:
        col_index = corr_var.columns.get_loc(colname)
        if(col_index < row_index_num and row[colname] > correlation_threshold and row.name != colname):
            #print '{},{} = {}'.format(row.name, colname, row[colname])
            #high_correlation.append({'var': (row.name, colname), 'var_corr': round(row[colname],3), 'var_na': na_count})
            high_correlation.append({'var1': row.name, 'var2': colname, 'var_corr': round(row[colname],3), 'var_na': na_count})

#highly correlated and with missing values
print 'highly correlated variables with missing value'
pp.pprint([x for x in high_correlation if x['var_na'] > 0])

print '\n'

print 'highly correlated variables without missing value'
pp.pprint([x for x in high_correlation if x['var_na'] == 0])


highly correlated variables with missing value
[{'var1': u'v12', 'var2': u'v10', 'var_corr': 0.912, 'var_na': 49851},
 {'var1': u'v17', 'var2': u'v4', 'var_corr': 0.776, 'var_na': 4},
 {'var1': u'v21', 'var2': u'v14', 'var_corr': 0.835, 'var_na': 49832},
 {'var1': u'v25', 'var2': u'v8', 'var_corr': 0.943, 'var_na': 49840},
 {'var1': u'v27', 'var2': u'v7', 'var_corr': 0.739, 'var_na': 500},
 {'var1': u'v28', 'var2': u'v18', 'var_corr': 0.701, 'var_na': 50675},
 {'var1': u'v32', 'var2': u'v13', 'var_corr': 0.737, 'var_na': 48619},
 {'var1': u'v32', 'var2': u'v15', 'var_corr': 0.908, 'var_na': 48619},
 {'var1': u'v37', 'var2': u'v1', 'var_corr': 0.894, 'var_na': 60110},
 {'var1': u'v41', 'var2': u'v29', 'var_corr': 0.904, 'var_na': 111},
 {'var1': u'v41', 'var2': u'v35', 'var_corr': 0.776, 'var_na': 111},
 {'var1': u'v43', 'var2': u'v7', 'var_corr': 0.725, 'var_na': 48624},
 {'var1': u'v43', 'var2': u'v26', 'var_corr': 0.903, 'var_na': 48624},
 {'var1': u'v44', 'var2': u'v4', 'var_corr': 

In [67]:
hi_cor_2 = [x for x in high_correlation if x['var_corr'] > 0.9]
hi_cor_2

[{'var1': u'v12', 'var2': u'v10', 'var_corr': 0.912, 'var_na': 49851},
 {'var1': u'v25', 'var2': u'v8', 'var_corr': 0.943, 'var_na': 49840},
 {'var1': u'v32', 'var2': u'v15', 'var_corr': 0.908, 'var_na': 48619},
 {'var1': u'v41', 'var2': u'v29', 'var_corr': 0.904, 'var_na': 111},
 {'var1': u'v43', 'var2': u'v26', 'var_corr': 0.903, 'var_na': 48624},
 {'var1': u'v46', 'var2': u'v8', 'var_corr': 0.975, 'var_na': 49836},
 {'var1': u'v46', 'var2': u'v25', 'var_corr': 0.982, 'var_na': 49836},
 {'var1': u'v49', 'var2': u'v41', 'var_corr': 0.904, 'var_na': 49832},
 {'var1': u'v53', 'var2': u'v11', 'var_corr': 0.956, 'var_na': 49796},
 {'var1': u'v54', 'var2': u'v25', 'var_corr': 0.968, 'var_na': 49832},
 {'var1': u'v54', 'var2': u'v46', 'var_corr': 0.919, 'var_na': 49832},
 {'var1': u'v55', 'var2': u'v33', 'var_corr': 0.943, 'var_na': 48619},
 {'var1': u'v60', 'var2': u'v26', 'var_corr': 0.981, 'var_na': 86},
 {'var1': u'v63', 'var2': u'v8', 'var_corr': 0.976, 'var_na': 49836},
 {'var1': u'v6

In [70]:
hi_cor_pd = pd.DataFrame(hi_cor_2)
print hi_cor_pd.shape
hi_cor_pd[1:20]

(54, 4)


,var1,var2,var_corr,var_na
1,v25,v8,0.943,49840
2,v32,v15,0.908,48619
3,v41,v29,0.904,111
4,v43,v26,0.903,48624
5,v46,v8,0.975,49836
6,v46,v25,0.982,49836
7,v49,v41,0.904,49832
8,v53,v11,0.956,49796
9,v54,v25,0.968,49832
10,v54,v46,0.919,49832


### 2.3. Transform Categorical data to continuous
* using get_dummies()

In [72]:
# Now convert all (except v22) variables to binary dummies
train_df_numeric = bnphelper.convert_to_numeric_dataframe(train_df, train_audit_text._a_variable)
train_df_numeric.head()

,ID,target,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v25,v26,v27,v28,v29,v32,v33,v34,v35,v36,v37,v39,v40,v41,v42,v43,v44,v45,v46,v48,v49,v50,v51,v53,v54,v55,v57,v58,v59,v60,v61,v63,v64,v65,v67,v68,v69,v70,v73,v76,v77,v78,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v108,v109,v111,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,...,v113_V,v113_W,v113_X,v113_Y,v113_Z,v125_A,v125_AA,v125_AB,v125_AC,v125_AD,v125_AE,v125_AF,v125_AG,v125_AH,v125_AI,v125_AJ,v125_AK,v125_AL,v125_AM,v125_AN,v125_AO,v125_AP,v125_AQ,v125_AR,v125_AS,v125_AT,v125_AU,v125_AV,v125_AW,v125_AX,v125_AY,v125_AZ,v125_B,v125_BA,v125_BB,v125_BC,v125_BD,v125_BE,v125_BF,v125_BG,v125_BH,v125_BI,v125_BJ,v125_BK,v125_BL,v125_BM,v125_BN,v125_BO,v125_BP,v125_BQ,v125_BR,v125_BS,v125_BT,v125_BU,v125_BV,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z,v129_0,v129_1,v129_2,v129_3,v129_4,v129_5,v129_6,v129_7,v129_8,v129_11
0,3,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,16.434108,6.085711,2.866830,11.636387,1.355013,8.571429,3.670350,0.106720,0.148883,18.869283,7.730923,XDX,-1.716131e-08,0.139412,1.720818,3.393503,0.590122,8.880867,1.083033,1.010829,7.270147,8.375452,11.326592,0.454546,4.012088,7.711453,7.653429,12.707581,2.015505,10.498338,9.848672,0.113561,12.171733,8.086643,0.899420,7.277792,16.747968,0.037096,1.299638,3.971118,0.529802,10.890984,1.588448,15.858152,0.153461,6.363189,18.303925,9.314079,15.231789,17.142857,11.784549,1.614988,2.230940,7.292418,8.571429,3.000000,7.528326,8.861647,0.649820,1.299638,1.707317,0.866426,9.551836,3.321300,0.095678,0.905342,0.442252,5.814018,3.517720,0.462019,7.436824,5.454545,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,1.574039,0.007294,12.579184,2.382692,3.930922,0.433213,15.634907,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,4,1,NaN,NaN,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,NaN,6.507647,NaN,11.636386,NaN,NaN,NaN,NaN,NaN,NaN,6.763110,GUV,NaN,3.056144,NaN,NaN,NaN,NaN,NaN,NaN,3.615077,NaN,14.579479,NaN,NaN,14.305766,NaN,NaN,NaN,NaN,NaN,2.449959,NaN,NaN,1.379210,NaN,NaN,1.129469,NaN,NaN,NaN,NaN,NaN,NaN,2.544736,NaN,NaN,NaN,NaN,NaN,12.053353,NaN,NaN,NaN,NaN,NaN,7.277655,3.430691,NaN,NaN,NaN,NaN,9.848004,NaN,2.678584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,1.505335,NaN,1.825361,4.247858,NaN,10.308044,NaN,NaN,10.595357,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,5,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,14.756098,6.384670,2.505589,9.603542,1.984127,5.882353,3.170847,0.244541,0.144258,17.952332,5.245035,FQ,-2.785053e-07,0.113997,2.244897,5.306122,0.836005,7.499999,1.454082,1.734693,4.043864,7.959184,12.730517,0.259740,7.378964,13.077201,6.173469,12.346939,2.926830,8.897561,5.343819,0.126035,12.711328,6.836734,0.604504,9.637627,15.102041,0.085573,0.765305,4.030613,4.277456,9.105481,2.151361,16.075602,0.123643,5.517949,16.377205,8.367347,11.040463,5.882353,8.460654,2.413618,1.963971,5.918368,11.764705,3.333334,10.194433,8.266200,1.530611,1.530613,2.429906,1.071429,8.447465,3.367346,0.111388,0.811447,0.271480,5.156559,4.214944,0.309657,5.663265,5.974026,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,1.668401,0.008275,11.67057

#### Save file

In [17]:
train_df_numeric_to_save = train_df_numeric

#bnp_converted_csv_path = "/Users/binlin/cbootcamp004_project/Project4-Machinelearning/Team_NA/bnp_converted.csv"
bnp_converted_csv_path = "/Users/sungmoon/Documents/Dropbox/DataScience/Week9/Project 4 - ML/BNP_cleaned2.csv"
train_df_numeric_to_save.to_csv(bnp_converted_csv_path, index=False)'''

In [73]:
##############################################
# Go to next section 
##############################################

In [59]:
##############################################
## Dropping BEFORE imputation
##############################################
'''
train_df = train_df.drop('v22', 1)
train_df = train_df.drop('v56', 1)
train_df = train_df.drop('v125', 1)
train_df.head()
'''
train_audit_text._a_variable

4        v3
23      v22
25      v24
31      v30
32      v31
39      v38
48      v47
53      v52
57      v56
63      v62
67      v66
72      v71
73      v72
75      v74
76      v75
80      v79
92      v91
108    v107
111    v110
113    v112
114    v113
126    v125
130    v129
Name: _a_variable, dtype: object

In [6]:
##############################################
## get_dummies
##############################################
def convert2numeric_msp_df(df, cate_variables):
    '''
    @Summary: convert the categorical variables to numeric variables by using dummies (binary).
     Old categorical variables will be dropped.
    @return: A copy of the old dataframe with new converted numeric variables. 
    '''
    # make a copy before creating dummies
    df_numeric = df.copy()

    # convert text-based columns to dummies (except v22)
    for var_name in cate_variables:
        if(var_name != 'v22' or var_name != 'v56' or var_name != 'v125'):        

            dummies = pd.get_dummies(df[var_name], prefix=var_name)
            #pp.pprint(dummies.head())

            #Drop the current variable, concat/append the dummy dataframe to the dataframe.
            #"""
            df_numeric = pd.concat([df_numeric.drop(var_name, 1), dummies.iloc[:,0:]], axis = 1)
            #"""
            #df_numeric = pd.concat([df_numeric, dummies.iloc[:,0:]], axis = 1)
            #df_numeric = pd.concat([df_numeric, dummies], axis = 1)

    #train_df_numeric.head()    
    #"""
    return df_numeric
    #"""
    #return "wow"

In [ ]:
train_df_numeric = convert2numeric_msp_df(train_df, train_audit_text._a_variable)
train_df_numeric.head()

In [26]:
##############################################
## Dropping After imputation (Maybe useless)
##############################################
#train_df = train_df.drop(['v22','v56','v125'], 1)
#'''
train_df_numeric = train_df_numeric.drop('v22', 1)
#train_df_numeric = train_df_numeric.drop('v56', 1)
#train_df_numeric = train_df_numeric.drop('v125', 1)
train_df_numeric.head()
#'''

,ID,target,v1,v2,v4,v5,v6,v7,v9,v14,v16,v18,v19,v21,v23,v27,v28,v35,v36,v37,v39,v40,v42,v44,v45,v48,v50,v51,v57,v58,v59,v61,v68,v69,v70,v78,v80,v81,v82,v84,v85,v87,v88,v90,v93,v94,v98,v99,v100,v101,v102,v103,v106,v114,v115,v117,v119,v120,v122,v123,v124,v126,v127,v130,v131,v3_A,v3_B,v3_C,v24_A,v24_B,v24_C,v24_D,v24_E,v30_A,v30_B,v30_C,v30_D,v30_E,v30_F,v30_G,v31_A,v31_B,v31_C,v38_0,v38_1,v38_10,v38_12,v38_2,v38_3,v38_4,v38_5,v38_6,v38_7,v38_8,v38_9,v47_A,v47_B,v47_C,v47_D,v47_E,v47_F,v47_G,v47_H,v47_I,v47_J,...,v113_V,v113_W,v113_X,v113_Y,v113_Z,v125_A,v125_AA,v125_AB,v125_AC,v125_AD,v125_AE,v125_AF,v125_AG,v125_AH,v125_AI,v125_AJ,v125_AK,v125_AL,v125_AM,v125_AN,v125_AO,v125_AP,v125_AQ,v125_AR,v125_AS,v125_AT,v125_AU,v125_AV,v125_AW,v125_AX,v125_AY,v125_AZ,v125_B,v125_BA,v125_BB,v125_BC,v125_BD,v125_BE,v125_BF,v125_BG,v125_BH,v125_BI,v125_BJ,v125_BK,v125_BL,v125_BM,v125_BN,v125_BO,v125_BP,v125_BQ,v125_BR,v125_BS,v125_BT,v125_BU,v125_BV,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z,v129_0,v129_1,v129_11,v129_2,v129_3,v129_4,v129_5,v129_6,v129_7,v129_8
0,3,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,9.999999,11.636387,8.571429,0.106720,0.148883,7.730923,-1.716131e-08,3.393503,0.590122,8.375452,11.326592,0.454546,4.012088,7.711453,12.707581,10.498338,9.848672,12.171733,0.899420,7.277792,3.971118,0.529802,10.890984,15.858152,15.231789,17.142857,11.784549,8.571429,3.000000,7.528326,8.861647,1.299638,1.707317,9.551836,3.321300,0.905342,5.814018,3.517720,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,12.579184,15.634907,2.857144,6.592012,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,1.804126,3.113719,0.636365,2.857144,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,4,1,NaN,NaN,NaN,9.191265,NaN,NaN,NaN,11.636386,NaN,NaN,NaN,6.763110,NaN,NaN,NaN,NaN,14.579479,NaN,NaN,14.305766,NaN,NaN,NaN,NaN,1.379210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.053353,NaN,NaN,7.277655,3.430691,NaN,NaN,9.848004,NaN,NaN,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,10.308044,NaN,10.595357,NaN,NaN,NaN,NaN,0.598896,NaN,NaN,NaN,NaN,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,5,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,12.666667,9.603542,5.882353,0.244541,0.144258,5.245035,-2.785053e-07,5.306122,0.836005,7.959184,12.730517,0.259740,7.378964,13.077201,12.346939,8.897561,5.343819,12.711328,0.604504,9.637627,4.030613,4.277456,9.105481,16.075602,11.040463,5.882353,8.460654,11.764705,3.333334,10.194433,8.266200,1.530613,2.429906,8.447465,3.367346,0.811447,5.156559,4.214944,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,11.670572,11.205561,12.941177,3.478911,-2.792745e-07,2.138728,9.333333,2.477596,0.013452,1.773709,3.922193,0.883118,1.176472,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,6,1,0.797415,8.304757,4.225930,11.627438,2.097700,1.987549,8.965516,14.094723,5.517242,1.224114,0.231630,7.517125,-4.805344e-07,2.303640,8.926662,8.898468,11.302795,0.433735,0.287322,11.523045,12.935823,12.708574,9.670823,12.194855,3.329176,4.780357,3.965517,1.732102,11.77

In [24]:
for f in train_df_numeric.columns:
    if f != 'v22':
        if train_df_numeric[f].dtype != 'float64':
            print f, "| ", train_df_numeric[f].dtype
        elif train_df_numeric[f].dtype == 'int64':
            print f#, "| ", train_df_numeric[f].dtype

ID |  int64
target |  int64


### 2.4. Imputation with mean

In [29]:
##############################################
## Imputation
##############################################

def impute_msp_df(train, test):

    df_imputed = train.copy()
    
    for f in df_imputed.columns:
        if f != 'ID' and f != 'target':

            if df_imputed[f].dtype == 'float64':  # All float64 are numerical
                # Fill NaN values with mean 
                df_imputed[f][np.isnan(df_imputed[f])] = df_imputed[f].mean()
                #test[f][np.isnan(test[f])] = test[f].mean()

            elif df_imputed[f].dtype == 'int64':  # All int64 are categorical
                # (a) Convert it to object ... (First solution)
                df_imputed[f] = df_imputed[f].astype(str)
                #test[f] = test[f].astype(str)
                
    return df_imputed

In [30]:
df_imputed = impute_msp_df(train_df_numeric, test)
df_imputed.head()

/Applications/DataScienceStudio.app/Contents/Resources/kit/dku-jupyter/packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,target,v1,v2,v4,v5,v6,v7,v9,v14,v16,v18,v19,v21,v23,v27,v28,v35,v36,v37,v39,v40,v42,v44,v45,v48,v50,v51,v57,v58,v59,v61,v68,v69,v70,v78,v80,v81,v82,v84,v85,v87,v88,v90,v93,v94,v98,v99,v100,v101,v102,v103,v106,v114,v115,v117,v119,v120,v122,v123,v124,v126,v127,v130,v131,v3_A,v3_B,v3_C,v24_A,v24_B,v24_C,v24_D,v24_E,v30_A,v30_B,v30_C,v30_D,v30_E,v30_F,v30_G,v31_A,v31_B,v31_C,v38_0,v38_1,v38_10,v38_12,v38_2,v38_3,v38_4,v38_5,v38_6,v38_7,v38_8,v38_9,v47_A,v47_B,v47_C,v47_D,v47_E,v47_F,v47_G,v47_H,v47_I,v47_J,...,v113_V,v113_W,v113_X,v113_Y,v113_Z,v125_A,v125_AA,v125_AB,v125_AC,v125_AD,v125_AE,v125_AF,v125_AG,v125_AH,v125_AI,v125_AJ,v125_AK,v125_AL,v125_AM,v125_AN,v125_AO,v125_AP,v125_AQ,v125_AR,v125_AS,v125_AT,v125_AU,v125_AV,v125_AW,v125_AX,v125_AY,v125_AZ,v125_B,v125_BA,v125_BB,v125_BC,v125_BD,v125_BE,v125_BF,v125_BG,v125_BH,v125_BI,v125_BJ,v125_BK,v125_BL,v125_BM,v125_BN,v125_BO,v125_BP,v125_BQ,v125_BR,v125_BS,v125_BT,v125_BU,v125_BV,v125_BW,v125_BX,v125_BY,v125_BZ,v125_C,v125_CA,v125_CB,v125_CC,v125_CD,v125_CE,v125_CF,v125_CG,v125_CH,v125_CI,v125_CJ,v125_CK,v125_CL,v125_D,v125_E,v125_F,v125_G,v125_H,v125_I,v125_J,v125_K,v125_L,v125_M,v125_N,v125_O,v125_P,v125_Q,v125_R,v125_S,v125_T,v125_U,v125_V,v125_W,v125_X,v125_Y,v125_Z,v129_0,v129_1,v129_11,v129_2,v129_3,v129_4,v129_5,v129_6,v129_7,v129_8
0,3,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,9.999999,11.636387,8.571429,0.106720,0.148883,7.730923,-1.716131e-08,3.393503,0.590122,8.375452,11.326592,0.454546,4.012088,7.711453,12.707581,10.498338,9.848672,12.171733,0.899420,7.277792,3.971118,0.529802,10.890984,15.858152,15.231789,17.142857,11.784549,8.571429,3.000000,7.528326,8.861647,1.299638,1.707317,9.551836,3.321300,0.905342,5.814018,3.517720,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,12.579184,15.634907,2.857144,6.592012,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,1.804126,3.113719,0.636365,2.857144,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,4,1,1.630686,7.464411,4.145098,9.191265,2.436402,2.483921,9.031859,11.636386,4.923222,0.841045,0.222300,6.763110,1.093088e+00,2.743454,5.093328,8.122387,14.579479,0.741471,1.237184,14.305766,12.924965,10.795169,9.142231,12.538022,1.379210,7.198159,4.077828,7.701653,10.587945,14.583035,17.564117,9.449335,12.053353,13.334482,2.209700,7.277655,3.430691,1.607956,2.822253,9.848004,1.924184,0.966913,5.475185,3.852883,8.303967,1.250721,12.091623,6.866414,2.890289,5.296716,11.791360,10.308044,10.548051,10.595357,3.168970e+00,1.291218,6.822439,3.549938,0.598896,1.672658,3.239542,1.925763,1.739389,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,5,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,12.666667,9.603542,5.882353,0.244541,0.144258,5.245035,-2.785053e-07,5.306122,0.836005,7.959184,12.730517,0.259740,7.378964,13.077201,12.346939,8.897561,5.343819,12.711328,0.604504,9.637627,4.030613,4.277456,9.105481,16.075602,11.040463,5.882353,8.460654,11.764705,3.333334,10.194433,8.266200,1.530613,2.429906,8.447465,3.367346,0.811447,5.156559,4.214944,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,11.670572,11.205561,12.941177,3.478911,-2.792745e-07,2.138728,9.333333,2.477596,0.013452,1.773709,3.922193,0.883118,1.176472,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,6,1,0.7974

#### Save

In [32]:
##############################################
## Save
##############################################
train_df_numeric_to_save = df_imputed

#bnp_converted_csv_path = "/Users/binlin/cbootcamp004_project/Project4-Machinelearning/Team_NA/bnp_converted.csv"
bnp_converted_csv_path = "/Users/sungmoon/Documents/Dropbox/DataScience/Week9/Project 4 - ML/BNP_hi_removed4.csv"
train_df_numeric_to_save.to_csv(bnp_converted_csv_pathth, index=False)

In [42]:
#

## 3. Data Without Highly-correlated variables 

In [16]:
train_df.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v9,v14,v16,v18,v19,v21,v23,v24,v27,v28,v30,v31,v35,v36,v37,v38,v39,v40,v42,v44,v45,v47,v48,v50,v51,v52,v57,v58,v59,v61,v62,v66,v68,v69,v70,v71,v72,v74,v75,v78,v79,v80,v81,v82,v84,v85,v87,v88,v90,v91,v93,v94,v98,v99,v100,v101,v102,v103,v106,v107,v110,v112,v113,v114,v115,v117,v119,v120,v122,v123,v124,v125,v126,v127,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,9.999999,11.636387,8.571429,0.106720,0.148883,7.730923,-1.716131e-08,C,3.393503,0.590122,C,A,8.375452,11.326592,0.454546,0,4.012088,7.711453,12.707581,10.498338,9.848672,C,12.171733,0.899420,7.277792,G,3.971118,0.529802,10.890984,15.858152,1,C,15.231789,17.142857,11.784549,F,1,B,D,8.571429,E,3.000000,7.528326,8.861647,1.299638,1.707317,9.551836,3.321300,0.905342,A,5.814018,3.517720,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,12.579184,E,B,O,NaN,15.634907,2.857144,6.592012,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,AU,1.804126,3.113719,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,NaN,11.636386,NaN,NaN,NaN,6.763110,NaN,C,NaN,NaN,C,A,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,E,NaN,1.379210,NaN,G,NaN,NaN,NaN,NaN,2,A,NaN,NaN,12.053353,F,2,B,D,NaN,D,NaN,7.277655,3.430691,NaN,NaN,9.848004,NaN,NaN,B,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,B,A,U,G,10.308044,NaN,10.595357,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,12.666667,9.603542,5.882353,0.244541,0.144258,5.245035,-2.785053e-07,E,5.306122,0.836005,NaN,A,7.959184,12.730517,0.259740,0,7.378964,13.077201,12.346939,8.897561,5.343819,C,12.711328,0.604504,9.637627,F,4.030613,4.277456,9.105481,16.075602,1,A,11.040463,5.882353,8.460654,B,3,B,B,11.764705,E,3.333334,10.194433,8.266200,1.530613,2.429906,8.447465,3.367346,0.811447,G,5.156559,4.214944,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,11.670572,C,B,S,NaN,11.205561,12.941177,3.478911,-2.792745e-07,2.138728,9.333333,2.477596,0.013452,AE,1.773709,3.922193,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,8.965516,14.094723,5.517242,1.224114,0.231630,7.517125,-4.805344e-07,D,2.303640,8.926662,C,B,8.898468,11.302795,0.433735,0,0.287322,11.523045,12.935823,12.708574,9.670823,C,12.194855,3.329176,4.780357,H,3.965517,1.732102,11.777912,15.927390,1,A,18.568129,9.425288,13.594728,F,2,B,D,13.448277,B,1.947261,4.797873,13.315819,1.379310,1.587045,10.747144,1.408046,1.042425,B,5.498902,3.423944,6.942002,1.334611,18.256352,8.507281,2.503055,4.872157,12.554274,B,B,J,NaN,13.777666,10.574713,4.949609,5.655086e-01,1.166281,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,10.991098,NaN,NaN,NaN,6.414567,NaN,E,NaN,NaN,NaN,A,NaN,NaN,NaN,0,NaN,10.138920,NaN,NaN,NaN,I,NaN,1.364536,NaN,H,NaN,NaN,NaN,NaN,1,C,NaN,NaN,NaN,F,1,B,D,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,A,T,G,14.097099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN,NaN,NaN,NaN


In [21]:
##############################################
## Dropping BEFORE imputation
##############################################
#''' These are all highly correlated variables (> .9)
train_df = train_df.drop(['v8','v10','v11','v12','v13','v15','v17',\
                          'v8','v10','v11','v12','v13','v15','v17',\
                          'v20','v25','v26','v29','v32','v33','v34',\
                          'v41','v43','v46','v49','v53','v54','v55',\
                          'v60','v63','v64','v65','v67','v73','v76','v77',\
                          'v83','v86','v89','v92','v95','v96','v97',\
                          'v104','v105','v108','v109','v111','v116',\
                          'v118','v121','v128'], 1)
train_df.head()
#'''
#train_audit_text._a_variable

,ID,target,v1,v2,v3,v4,v5,v6,v7,v9,v14,v16,v18,v19,v21,v22,v23,v24,v27,v28,v30,v31,v35,v36,v37,v38,v39,v40,v42,v44,v45,v47,v48,v50,v51,v52,v56,v57,v58,v59,v61,v62,v66,v68,v69,v70,v71,v72,v74,v75,v78,v79,v80,v81,v82,v84,v85,v87,v88,v90,v91,v93,v94,v98,v99,v100,v101,v102,v103,v106,v107,v110,v112,v113,v114,v115,v117,v119,v120,v122,v123,v124,v125,v126,v127,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,9.999999,11.636387,8.571429,0.106720,0.148883,7.730923,XDX,-1.716131e-08,C,3.393503,0.590122,C,A,8.375452,11.326592,0.454546,0,4.012088,7.711453,12.707581,10.498338,9.848672,C,12.171733,0.899420,7.277792,G,DI,3.971118,0.529802,10.890984,15.858152,1,C,15.231789,17.142857,11.784549,F,1,B,D,8.571429,E,3.000000,7.528326,8.861647,1.299638,1.707317,9.551836,3.321300,0.905342,A,5.814018,3.517720,8.877414,1.191337,19.470199,8.389237,2.757375,4.374296,12.579184,E,B,O,NaN,15.634907,2.857144,6.592012,-6.297423e-07,1.059603,8.000000,1.989780,0.035754,AU,1.804126,3.113719,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,NaN,11.636386,NaN,NaN,NaN,6.763110,GUV,NaN,C,NaN,NaN,C,A,NaN,14.579479,NaN,0,NaN,14.305766,NaN,NaN,NaN,E,NaN,1.379210,NaN,G,DY,NaN,NaN,NaN,NaN,2,A,NaN,NaN,12.053353,F,2,B,D,NaN,D,NaN,7.277655,3.430691,NaN,NaN,9.848004,NaN,NaN,B,NaN,NaN,8.303967,NaN,NaN,NaN,NaN,NaN,NaN,B,A,U,G,10.308044,NaN,10.595357,NaN,NaN,NaN,NaN,0.598896,AF,NaN,NaN,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,12.666667,9.603542,5.882353,0.244541,0.144258,5.245035,FQ,-2.785053e-07,E,5.306122,0.836005,NaN,A,7.959184,12.730517,0.259740,0,7.378964,13.077201,12.346939,8.897561,5.343819,C,12.711328,0.604504,9.637627,F,AS,4.030613,4.277456,9.105481,16.075602,1,A,11.040463,5.882353,8.460654,B,3,B,B,11.764705,E,3.333334,10.194433,8.266200,1.530613,2.429906,8.447465,3.367346,0.811447,G,5.156559,4.214944,11.588858,0.841837,15.491329,5.879353,3.292788,5.924457,11.670572,C,B,S,NaN,11.205561,12.941177,3.478911,-2.792745e-07,2.138728,9.333333,2.477596,0.013452,AE,1.773709,3.922193,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,8.965516,14.094723,5.517242,1.224114,0.231630,7.517125,ACUE,-4.805344e-07,D,2.303640,8.926662,C,B,8.898468,11.302795,0.433735,0,0.287322,11.523045,12.935823,12.708574,9.670823,C,12.194855,3.329176,4.780357,H,BW,3.965517,1.732102,11.777912,15.927390,1,A,18.568129,9.425288,13.594728,F,2,B,D,13.448277,B,1.947261,4.797873,13.315819,1.379310,1.587045,10.747144,1.408046,1.042425,B,5.498902,3.423944,6.942002,1.334611,18.256352,8.507281,2.503055,4.872157,12.554274,B,B,J,NaN,13.777666,10.574713,4.949609,5.655086e-01,1.166281,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,10.991098,NaN,NaN,NaN,6.414567,HIT,NaN,E,NaN,NaN,NaN,A,NaN,NaN,NaN,0,NaN,10.138920,NaN,NaN,NaN,I,NaN,1.364536,NaN,H,NaN,NaN,NaN,NaN,NaN,1,C,NaN,NaN,NaN,F,1,B,D,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,A,T,G,14.097099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z,NaN,NaN,0,NaN,NaN


### 3.1. Imputation with Mean

In [77]:
########################################
## Omitted (same process)
########################################